# 시작

In [13]:
import xarray as xr
import numpy as np
import his_utils
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from multiprocessing import Pool

# matplotlib 백엔드를 Agg로 설정 (GUI 없이 실행)
plt.switch_backend('Agg')

In [14]:
# 현재 시간의 데이터를 선택합니다.

def plot_CPU(args):
    diff, mean_ERA, mean_google, time_index, target_var, param, weights = args

    if target_var not in ['land_sea_mask', 'geopotential_at_surface']:
        current_diff = diff.isel(time=time_index)
    else:
        current_diff = diff
    
    # 플롯 생성
    fig = plt.figure(figsize=(20,10))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

    # 메쉬 그리드 생성
    lon, lat = np.meshgrid(current_diff.lon, current_diff.lat)

    weighted = current_diff.weighted(weights)
    mean_val = weighted.mean(('lat', 'lon'))
    max_val = np.abs(current_diff).max().values
    
    # TwoSlopeNorm을 사용하여 0을 중심으로 대칭적인 컬러맵 생성
    norm = TwoSlopeNorm(vmin=-max_val, vcenter=0, vmax=max_val)
    # 차이 데이터를 지도에 그립니다.
    im = ax.pcolormesh(lon, lat, current_diff.squeeze().values, 
                    transform=ccrs.PlateCarree(), 
                    cmap='RdBu_r',
                    norm=norm,
                    shading='auto')
    # 컬러바 추가
    if target_var == 'total_precipitation_6hr':
        cbar = plt.colorbar(im, ax=ax, extend='max', 
                            label=f'{target_var} difference (mm)')
    elif target_var == 'mean_sea_level_pressure':
        cbar = plt.colorbar(im, ax=ax, extend='both', 
                            label=f'{target_var} difference (hPa)')
    else:
        cbar = plt.colorbar(im, ax=ax, extend='both', 
                        label=f'{target_var} difference {ERA5[target_var].units}')
    # 지도 특성 추가
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS)
    ax.gridlines(draw_labels=True)

    # 제목 설정 (시간 정보 포함)
    plt.title(f'{target_var} Diff at t:{time_index} \ndiff mean: {mean_val.values}, diff max: {max_val:.10f}\n(ERA5 mean:{mean_ERA[time_index]})\n(google mean:{mean_google[time_index]})')   
    # plt.title(f"{param}")
    ax.set_global()
    plt.savefig(f'figure/{target_var}{param}_{time_index}.png', dpi=300, bbox_inches='tight')
    plt.close("all")

In [15]:
param_list = ['RR', 'LR', 'RL', 'LL']

for param in param_list:
    ERA5 = xr.open_dataset(f'testdata/2022-01-01{param}.nc')
    google = xr.open_dataset('testdata/source-era5_date-2022-01-01_res-0.25_levels-37_steps-12.nc').drop_vars('toa_incident_solar_radiation')

    tp = 'total_precipitation_6hr'
    ERA5[tp] = ERA5[tp] * 1000
    google[tp] = google[tp] * 1000

    weights = np.cos(np.deg2rad(ERA5.lat))
    weights.name = "weights"
    
    mean_ERA = {}
    mean_google = {}
    
    for t in range(0, 14):
        mean_ERA[t]=ERA5[tp].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
        mean_google[t]=google[tp].isel(time=t).weighted(weights).mean(('lat', 'lon')).values    

    for target_var in ERA5.data_vars:
        # 지금부턴 tp만 비교하자
        if target_var != 'total_precipitation_6hr':
            continue
        
        if 'level' in ERA5[target_var].dims:
            diff = (ERA5[target_var] - google[target_var]).mean(dim='level')
        else:
            diff = ERA5[target_var] - google[target_var]
    
        # 시간 차원에 대해 반복합니다.
        # for time_index in range(diff.time.size):
        arg_list = [(diff, mean_ERA, mean_google, time_index, target_var, param, weights)
            for time_index in range(diff.time.size)]

        with Pool() as pool:
            pool.map(plot_CPU, arg_list)



##test 07-26

In [11]:
weights = np.cos(np.deg2rad(ERA5.lat))
weights.name = "weights"

mean_ERA = {}
mean_google = {}

for t in range(0, 14):
    mean_ERA[t]=ERA5[tp].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
    mean_google[t]=ERA5[tp].isel(time=t).weighted(weights).mean(('lat', 'lon')).values

# for var in ERA5.data_vars:
#     if var not in ['land_sea_mask', 'geopotential_at_surface']:
#         if 'level' in ERA5[var].dims:
#             mean_ERA[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).mean(dim='level').values
#             mean_google[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).mean(dim='level').values
#         else:
#             mean_ERA[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
#             mean_google[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
        
#     else:
#         mean_ERA[var]=ERA5[var].weighted(weights).mean(('lat', 'lon')).values
#         mean_google[var]=ERA5[var].weighted(weights).mean(('lat', 'lon')).values

In [12]:
for target_var in ERA5.data_vars:
    # 지금부턴 tp만 비교하자
    if target_var != 'total_precipitation_6hr':
        continue
    
    if 'level' in ERA5[target_var].dims:
        diff = (ERA5[target_var] - google[target_var]).mean(dim='level')
    else:
        diff = ERA5[target_var] - google[target_var]
  
    # 시간 차원에 대해 반복합니다.
    for time_index in range(diff.time.size):
        # 현재 시간의 데이터를 선택합니다.
        if target_var not in ['land_sea_mask', 'geopotential_at_surface']:
            current_diff = diff.isel(time=time_index)
        else:
            current_diff = diff
        
        # 플롯 생성
        fig = plt.figure(figsize=(20,10))
        ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
    
        # 메쉬 그리드 생성
        lon, lat = np.meshgrid(current_diff.lon, current_diff.lat)
    
        weighted = current_diff.weighted(weights)
        mean_val = weighted.mean(('lat', 'lon'))
        max_val = np.abs(current_diff).max().values
        
        # TwoSlopeNorm을 사용하여 0을 중심으로 대칭적인 컬러맵 생성
        norm = TwoSlopeNorm(vmin=-max_val, vcenter=0, vmax=max_val)
        # 차이 데이터를 지도에 그립니다.
        im = ax.pcolormesh(lon, lat, current_diff.squeeze().values, 
                           transform=ccrs.PlateCarree(), 
                           cmap='RdBu_r',
                           norm=norm,
                           shading='auto')
        # 컬러바 추가
        if target_var == 'total_precipitation_6hr':
            cbar = plt.colorbar(im, ax=ax, extend='max', 
                                label=f'{target_var} difference (mm)')
        elif target_var == 'mean_sea_level_pressure':
            cbar = plt.colorbar(im, ax=ax, extend='both', 
                                label=f'{target_var} difference (hPa)')
        else:
            cbar = plt.colorbar(im, ax=ax, extend='both', 
                            label=f'{target_var} difference {ERA5[target_var].units}')
        # 지도 특성 추가
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS)
        ax.gridlines(draw_labels=True)
    
        # 제목 설정 (시간 정보 포함)
        plt.title(f'{target_var} Diff at t:{time_index} \ndiff mean: {mean_val.values}, diff max: {max_val:.10f}\n(ERA5 mean:{mean_ERA[time_index]})\n(google mean:{mean_google[time_index]})')   
        # plt.title(f"{param}")
        ax.set_global()
        plt.savefig(f'figure/{target_var}{param}_{time_index}.png', dpi=300, bbox_inches='tight')
        plt.close("all")


(array([1.4491691e+07, 3.5535000e+04, 5.3590000e+03, 1.6600000e+03,
        6.2600000e+02, 2.9200000e+02, 1.2800000e+02, 4.2000000e+01,
        2.1000000e+01, 6.0000000e+00]),
 array([-1.11758709e-05,  1.59172487e+01,  3.18345089e+01,  4.77517700e+01,
         6.36690292e+01,  7.95862885e+01,  9.55035477e+01,  1.11420815e+02,
         1.27338074e+02,  1.43255325e+02,  1.59172592e+02]),
 <BarContainer object of 10 artists>)

True